In [1]:
from pathlib import Path

import numpy as np
import easyidp as idp
import geopandas as gpd

In [2]:
NAS_ROOT = Path("/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli")

dom_root = NAS_ROOT / "03_metashape.psx" / "outputs"

SHP_DIR = NAS_ROOT / "11_root_position" / "shapefiles"

SEG_DIR = NAS_ROOT / "12_head_seg" / "head_neighbor"

In [3]:
def find_dom_files(base_path: Path) -> dict:
    """
    遍历指定路径下的时间序列子文件夹，构建一个字典。
    - Key: 子文件夹的名称 (e.g., "20250918")
    - Value: 子文件夹中 _dom.tif 文件的完整路径对象
    :param base_path: 要遍历的根目录 (e.g., .../outputs)
    :return: 一个包含文件夹名称和对应 dom.tif 路径的字典
    """
    dom_files_dict = {}
    
    # 检查基础路径是否存在
    if not base_path.exists() or not base_path.is_dir():
        print(f"错误: 路径 '{base_path}' 不存在或不是一个目录。")
        return dom_files_dict
    # 遍历 base_path 下的所有条目（文件和文件夹）
    for sub_dir in base_path.iterdir():
        # 确保我们只处理子目录
        if sub_dir.is_dir():
            # 使用 glob 查找目录中符合条件的 dom.tif 文件
            # 模式 f"{sub_dir.name}_dom.tif" 能精确匹配，例如在 "20250918" 文件夹下查找 "20250918_dom.tif"
            # 使用 list() 将生成器结果转换为列表
            found_files = list(sub_dir.glob(f"{sub_dir.name}_dom.tif"))
            
            # 如果找到了唯一的一个文件
            if len(found_files) == 1:
                # 文件夹名作为 key，文件的完整路径作为 value
                dom_files_dict[sub_dir.name] = found_files[0]
            elif len(found_files) > 1:
                print(f"警告: 在目录 '{sub_dir}' 中找到多个匹配的 dom.tif 文件，已跳过。")
            # 如果没找到文件，则不执行任何操作，自然跳过
    return dom_files_dict

def find_dsm_files(base_path: Path) -> dict:
    """
    遍历指定路径下的时间序列子文件夹，构建一个字典。
    - Key: 子文件夹的名称 (e.g., "20250918")
    - Value: 子文件夹中 _dom.tif 文件的完整路径对象
    :param base_path: 要遍历的根目录 (e.g., .../outputs)
    :return: 一个包含文件夹名称和对应 dom.tif 路径的字典
    """
    dom_files_dict = {}
    
    # 检查基础路径是否存在
    if not base_path.exists() or not base_path.is_dir():
        print(f"错误: 路径 '{base_path}' 不存在或不是一个目录。")
        return dom_files_dict
    # 遍历 base_path 下的所有条目（文件和文件夹）
    for sub_dir in base_path.iterdir():
        # 确保我们只处理子目录
        if sub_dir.is_dir():
            # 使用 glob 查找目录中符合条件的 dom.tif 文件
            # 模式 f"{sub_dir.name}_dom.tif" 能精确匹配，例如在 "20250918" 文件夹下查找 "20250918_dom.tif"
            # 使用 list() 将生成器结果转换为列表
            found_files = list(sub_dir.glob(f"{sub_dir.name}_dsm.tif"))
            
            # 如果找到了唯一的一个文件
            if len(found_files) == 1:
                # 文件夹名作为 key，文件的完整路径作为 value
                dom_files_dict[sub_dir.name] = found_files[0]
            elif len(found_files) > 1:
                print(f"警告: 在目录 '{sub_dir}' 中找到多个匹配的 dsm.tif 文件，已跳过。")
            # 如果没找到文件，则不执行任何操作，自然跳过
    return dom_files_dict

In [4]:
dom_dict = find_dom_files(dom_root)
dsm_dict = find_dsm_files(dom_root)

In [5]:
dom_dict

{'20250918': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20250918/20250918_dom.tif'),
 '20251002': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20251002/20251002_dom.tif'),
 '20251010': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20251010/20251010_dom.tif'),
 '20251016': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20251016/20251016_dom.tif'),
 '20251020': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20251020/20251020_dom.tif'),
 '20251022': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20251022/20251022_dom.tif'),
 '20251024': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20251024/20251024_dom.tif'),
 '20251027': PosixPath('/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_met

In [6]:
center_points_path = SHP_DIR / "2_ordered_center_points.shp"

In [7]:
center_points = gpd.read_file(center_points_path)
center_points

,name,geometry
0,110,POINT (368166.107 3956030.317)
1,217,POINT (368165.404 3956030.379)
2,216,POINT (368165.31 3956030.792)
3,106,POINT (368165.746 3956032.073)
4,109,POINT (368166.035 3956030.729)
...,...,...
2617,2514,POINT (368138.561 3956075.21)
2618,2405,POINT (368139.422 3956074.966)
2619,2407,POINT (368139.582 3956074.075)
2620,2404,POINT (368139.337 3956075.462)


In [8]:
coords_df = center_points.get_coordinates().values

coords_df

array([[ 368166.10748715, 3956030.31670528],
       [ 368165.4042752 , 3956030.37862227],
       [ 368165.30979474, 3956030.79158518],
       ...,
       [ 368139.58157867, 3956074.07510673],
       [ 368139.33700166, 3956075.46237242],
       [ 368138.67126428, 3956074.75033545]], shape=(2622, 2))

In [9]:
buffer = 0.2 #m

In [10]:
head_id = 110

In [11]:
head_ctr = coords_df[0,:]
head_ctr

array([ 368166.10748715, 3956030.31670528])

In [12]:
head_x, head_y = head_ctr

x_min = head_x - buffer
x_max = head_x + buffer

y_min = head_y - buffer
y_max = head_y + buffer

roi_buffer = np.asarray([
    [x_min, y_min],
    [x_max, y_min],
    [x_max, y_max],
    [x_min, y_max],
    [x_min, y_min],
])

In [13]:
roi_buffer

array([[ 368165.90748715, 3956030.11670528],
       [ 368166.30748715, 3956030.11670528],
       [ 368166.30748715, 3956030.51670528],
       [ 368165.90748715, 3956030.51670528],
       [ 368165.90748715, 3956030.11670528]])

In [36]:
str(dom_dict['20250918']).replace("_dom", "_dsm")

'/home/crest/w/hwang_Pro/data/202509_tanashi_broccoli/03_metashape.psx/outputs/20250918/20250918_dsm.tif'

In [16]:
roi = idp.ROI()

In [17]:
roi[str(head_id)] = roi_buffer

In [18]:
dom = idp.GeoTiff(dom_dict['20250918'])

In [19]:
roi.crs = dom.crs

In [20]:
roi

<easyidp.ROI> with 1 items
[0]	110
array([[ 368165.90748715, 3956030.11670528],
       [ 368166.30748715, 3956030.11670528],
       [ 368166.30748715, 3956030.51670528],
       [ 368165.90748715, 3956030.51670528],
       [ 368165.90748715, 3956030.11670528]], shape=(5, 2))

In [50]:
ms_project = dom_root = NAS_ROOT / "03_metashape.psx" / "25Autumn_rgb.psx"

In [51]:
ms = idp.Metashape(ms_project)
ms

/home/crest/e/NasBackup/hwang_Pro/jupyter/19_UAVBroccoli_2025/.venv/lib/python3.12/site-packages/easyidp/metashape.py:320: UserWarning: The project has [25] chunks, however no chunk_id has been specified, open the first chunk [0] '20250911' by default.
  warnings.warn(


<'25Autumn_rgb.psx' easyidp.Metashape object with 25 active chunks>

  id  label
----  --------
-> 0  20250911
   1  20250912
   2  20250916
   3  20250918
   4  20251002
   5  20251010
   6  20251016
   7  20251020
   8  20251022
   9  20251024
  10  20251027
  11  20251031
  12  20251104
  13  20251105
  14  20251107
  15  20251110
  16  20251112
  17  20251029
  18  20251114
  19  20251117
  20  20251119
  23  20251121
  22  20251126
  24  20251128
  25  20251201

In [54]:
ms.open_chunk("20251128")

ms

<'25Autumn_rgb.psx' easyidp.Metashape object with 25 active chunks>

   id  label
-----  --------
    0  20250911
    1  20250912
    2  20250916
    3  20250918
    4  20251002
    5  20251010
    6  20251016
    7  20251020
    8  20251022
    9  20251024
   10  20251027
   11  20251031
   12  20251104
   13  20251105
   14  20251107
   15  20251110
   16  20251112
   17  20251029
   18  20251114
   19  20251117
   20  20251119
   23  20251121
   22  20251126
-> 24  20251128
   25  20251201

In [69]:
for date, dom_path in dom_dict.items():
    dom = idp.GeoTiff(dom_path)
    dsm = idp.GeoTiff(str(dom_path).replace("_dom", "_dsm"))

    roi = idp.ROI()
    roi[0] = roi_buffer
    roi.crs = dom.crs
    roi.item_label = {str(head_id): 0}

    roi.get_z_from_dsm(dsm)

    ms.open_chunk(date)

    img_dict_ms = ms.back2raw(roi)

    img_dict_sort = ms.sort_img_by_distance(
        img_dict_ms, roi,
        distance_thresh=2,  # distance threshold is 2m
        num=1,   # only keep 1 closest images
        save_folder=SEG_DIR
    )

    # ms.show_roi_on_img(img_dict_sort, "110", save_as=SEG_DIR / f"{head_id}_{date}_vis.png")

    # break
    

Read z values of roi from DSM [20250918_dsm.tif]:   0%|          | 0/1 [00:00<?, ?it/s]

Filter by distance to ROI: 100%|██████████| 1/1 [00:00<00:00, 4346.43it/s]


Optimising data structures of produced results, this may take some time...


Filter by distance to ROI: 100%|██████████| 1/1 [00:00<00:00, 4928.68it/s]


Optimising data structures of produced results, this may take some time...


...

Filter by distance to ROI: 100%|██████████| 1/1 [00:00<00:00, 4319.57it/s]


Optimising data structures of produced results, this may take some time...


Filter by distance to ROI: 100%|██████████| 1/1 [00:00<00:00, 4882.78it/s]


Optimising data structures of produced results, this may take some time...


Processing image [DJI_20251201121756_0306]: 100%|██████████| 1/1 [00:00<00:00,  2.25it/s]


In [56]:
img_dict_ms

{0: {'DJI_20250918120927_0070': array([[4392.5449167 , 2988.0362205 ],
         [5941.62157099, 3662.83396925],
         [5652.79495031, 3496.20218314],
         [4126.27186873, 2868.31564401],
         [4392.5449167 , 2988.0362205 ]]),
  'DJI_20250918120928_0071': array([[2498.94949376, 2302.6581946 ],
         [3806.70969537, 2753.15203526],
         [3665.68063939, 2714.35591767],
         [2365.6864732 , 2299.17160633],
         [2498.94949376, 2302.6581946 ]]),
  'DJI_20250918120930_0072': array([[1250.47099678, 2160.8136007 ],
         [2394.71269283, 2437.24338665],
         [2380.77388802, 2473.93385295],
         [1230.87070068, 2225.40239544],
         [1250.47099678, 2160.8136007 ]]),
  'DJI_20250918120932_0073': array([[ 551.45849141, 2327.45873912],
         [1594.97394529, 2474.41223162],
         [1687.81019296, 2543.50192887],
         [ 627.09457956, 2420.68618885],
         [ 551.45849141, 2327.45873912]]),
  'DJI_20250918120933_0074': array([[ 255.22406323, 2648.3789